In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/test_statement_section.csv')
test_df

,statement,section,label
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction
...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction


In [3]:
hypothesis_lst=test_df['statement'].values.tolist()
len(hypothesis_lst)

500

In [4]:
evidence_lst=test_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

500

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

500

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in test_df[['statement','section']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

/home/ai/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.473456,0.627338,0.489888,0.872,0.482
1,0.1,0.473456,0.473988,0.457249,0.492,0.454
2,0.2,0.473456,0.283333,0.463636,0.204,0.484
3,0.3,0.473456,0.155932,0.511111,0.092,0.502
4,0.4,0.473456,0.037594,0.312500,0.020,0.488
5,0.5,0.473456,0.007905,0.333333,0.004,0.498
6,0.6,0.473456,0.007937,0.500000,0.004,0.500
7,0.7,0.473456,0.000000,0.000000,0.000,0.498
8,0.8,0.473456,0.000000,0.000000,0.000,0.500
9,0.9,0.473456,0.000000,0.000000,0.000,0.500
